# Itinerary Basics

Basic report on stage itinerary - stage name, distance etc.

In [1]:
import notebookimport

In [2]:
import os
import sqlite3
import pandas as pd

In [3]:
#!pip3 install tracery
#!pip3 install inflect

In [4]:
if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='australia18.db'
    conn = sqlite3.connect(dbname)
    rally='Australia'
    rebase = 'PAD'
    rebase = ''

In [5]:
#!/Users/ajh59/anaconda3/bin/pip install tracery
import tracery
from tracery.modifiers import base_english
from inflectenglish import inflect_english
from pytracery_logic import pytracery_logic

def pandas_row_mapper(row, rules, root,  modifiers=base_english):
    ''' Generate text from a single row of dataframe using a tracery grammar. '''
    row=row.to_dict()
    rules=rules.copy()

    for k in row:
        rules[k] = str(row[k])
        grammar = tracery.Grammar(rules)
        if modifiers is not None:
            if isinstance(modifiers,list):
                for modifier in modifiers:
                    grammar.add_modifiers(modifier)
            else:
                grammar.add_modifiers(modifiers)

    return grammar.flatten(root)

def traceryparse(rules, root, modifiers=base_english):
    ''' Create a tracery grammar from a rules set and set of modifiers and apply it to tracery template string. '''
    grammar = tracery.Grammar(rules)
    if modifiers is not None:
        if isinstance(modifiers,list):
            for modifier in modifiers:
                grammar.add_modifiers(modifier)
        else:
            grammar.add_modifiers(modifiers)
    
    return grammar.flatten(root)
    

## Rally Summary

In [6]:
def dbGetRallySummary(rally, year=2018):
    ''' Create a dataframe containing summary details of a specified rally. '''
    q='''
    SELECT o.*, t.totalDistance FROM 
        (SELECT ce.`country.name`, ce.startDate, ce.finishDate, ce.name, ce.`country.iso3`, ce.surfaces,
            COUNT(*) numOfStages, SUM(itc.distance) AS compDistanceKm
            FROM itinerary_controls itc
            JOIN championship_events ce ON itc.eventId=ce.eventId
            WHERE ce.`country.name`="{rally}" AND type='StageStart' AND strftime('%Y', startDate)='{year}' ) AS o
        JOIN (SELECT ce.`country.name`, SUM(itc.distance) totalDistance FROM itinerary_controls itc
                JOIN championship_events ce ON itc.eventId=ce.eventId
                WHERE ce.`country.name`="{rally}") AS t ON o.`country.name` = t.`country.name`
    '''.format(rally=rally, year=year)
    
    rallydetails = pd.read_sql(q,conn)
    
    return rallydetails


In [7]:
if __name__=='__main__':
    rs = dbGetRallySummary(rally)
    display(rs)
    display(rs.dtypes)

,country.name,startDate,finishDate,name,country.iso3,surfaces,numOfStages,compDistanceKm,totalDistance
0,Australia,2018-11-15,2018-11-18,Kennards Hire Rally Australia,AUS,Gravel,24,318.64,1017.07


country.name       object
startDate          object
finishDate         object
name               object
country.iso3       object
surfaces           object
numOfStages         int64
compDistanceKm    float64
totalDistance     float64
dtype: object

Tracery rules specify an emergent story template.

In [8]:
rules = {'origin': "#name# (#startDate# to #finishDate#) #stages#. #dist#. #surface#.",
         'stages': "runs over #numOfStages# competitive special stages",
         'dist': "The distance covered on the special stages is #compDistanceKm.round#km, with an overall rally distance of #totalDistance.round#km",
         'surface':"The special stage surface type is predominantly #surfaces#",
        }

In [9]:
rs['report'] = rs.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english, pytracery_logic]), axis=1)

In [10]:
rs[rs['country.name']==rally]['report'].iloc[0]

'Kennards Hire Rally Australia (2018-11-15 to 2018-11-18) runs over 24 competitive special stages. The distance covered on the special stages is 318.64km, with an overall rally distance of 1017.07km. The special stage surface type is predominantly Gravel.'

In [11]:
basepath = 'report'
if not os.path.exists(basepath):
    os.makedirs(basepath)

In [12]:
README='''# Rally Report - {}

*This report is unofficial and is not associated in any way with the Fédération Internationale de l’Automobile (FIA) or WRC Promoter GmbH.*


{}
'''.format(rally, rs[rs['country.name']==rally]['report'].iloc[0])

with open('{}/README.md'.format(basepath), 'w') as out_file:
    out_file.write(README)

In [13]:
import inflect
p=inflect.engine()

In [14]:
rs['startDate']=pd.to_datetime(rs['startDate'])
rs['startDate'].loc[0].strftime("%A %d,  %B %Y")

'Thursday 15,  November 2018'

In [15]:
p.number_to_words(p.ordinal((int(rs['startDate'].loc[0].strftime("%d")))))

'fifteenth'

## Itinerary Items

In [16]:
year=2018

In [17]:
q='''
    SELECT *
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND strftime('%Y', startDate)='{year}' 
            AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally, year=year)
xx =pd.read_sql(q,conn)
xx.columns

Index(['code', 'controlId', 'controlPenalties', 'distance', 'eventId',
       'firstCarDueDateTime', 'firstCarDueDateTimeLocal', 'location',
       'stageId', 'status', 'targetDuration', 'targetDurationMs',
       'timingPrecision', 'type', 'itineraryLegId',
       'itinerarySections.itinerarySectionId', 'roundingPolicy', 'categories',
       'clerkOfTheCourse', 'country.countryId', 'country.iso2', 'country.iso3',
       'country.name', 'countryId', 'eventId', 'finishDate', 'location',
       'mode', 'name', 'organiserUrl', 'slug', 'startDate', 'stewards',
       'surfaces', 'templateFilename', 'timeZoneId', 'timeZoneName',
       'timeZoneOffset', 'trackingEventId', 'itineraryLegId',
       'itinerarySectionId', 'name', 'order', 'itineraryId', 'itineraryLegId',
       'legDate', 'name', 'order', 'startListId', 'status'],
      dtype='object')

In [18]:
xx.head()

,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,status,...,itinerarySectionId,name,order,itineraryId,itineraryLegId,legDate,name,order,startListId,status
0,TC0,3503,All,NaN,38,2018-11-15T20:15:00,2018-11-16T07:15:00+11:00,Coffs Start,-1,Completed,...,338,Section 1,1,125,146,2018-11-16,Friday 16th November,1,242,Completed
1,TC1,3558,All,31.68,38,2018-11-15T21:00:00,2018-11-16T08:00:00+11:00,Orara East I,838,Completed,...,338,Section 1,1,125,146,2018-11-16,Friday 16th November,1,242,Completed
2,SS1,3557,None,8.77,38,2018-11-15T21:03:00,2018-11-16T08:03:00+11:00,Orara East I,838,Completed,...,338,Section 1,1,125,146,2018-11-16,Friday 16th November,1,242,Completed
3,TC2,3555,All,23.95,38,2018-11-15T21:40:00,2018-11-16T08:40:00+11:00,Coldwater I,840,Completed,...,338,Section 1,1,125,146,2018-11-16,Friday 16th November,1,242,Completed
4,SS2,3554,None,14.12,38,2018-11-15T21:43:00,2018-11-16T08:43:00+11:00,Coldwater I,840,Completed,...,338,Section 1,1,125,146,2018-11-16,Friday 16th November,1,242,Completed


In [19]:
import datetime
t=datetime.time(abs(int(360/60)), 360 % 60)
dt = datetime.datetime.combine(datetime.date.today(), t)
dt.isoformat()

'2018-11-18T06:00:00'

In [20]:
t = datetime.datetime.combine(datetime.date.today(),datetime.time(0))
(t  + datetime.timedelta( minutes=-359)).isoformat()

'2018-11-17T18:01:00'

In [21]:
xx[:2][['timeZoneId','timeZoneOffset']]

,timeZoneId,timeZoneOffset
0,AUS Eastern Standard Time,660
1,AUS Eastern Standard Time,660


In [22]:
def dbGetTimeControls(rally, year=2018):
    ''' Get dataframe containing time control details for a specified rally. '''
    q='''
    SELECT il.name AS date, itc.*, ce.timeZoneOffset,
         isc.itinerarySectionId, isc.name AS section, isc.`order`
    FROM itinerary_controls itc
    JOIN championship_events ce ON itc.eventId=ce.eventId
    JOIN itinerary_sections isc ON itc.`itinerarySections.itinerarySectionId`=isc.itinerarySectionId
    JOIN itinerary_legs il ON isc.itineraryLegId=il.itineraryLegId
    WHERE ce.`country.name`="{rally}" AND strftime('%Y', startDate)='{year}'
            AND firstCarDueDateTimeLocal NOT NULL ORDER BY firstCarDueDateTimeLocal 
    '''.format(rally=rally, year=year)
    time_controls = pd.read_sql(q,conn)
    time_controls['firstCarDueDateTimeLocal']=pd.to_datetime(time_controls['firstCarDueDateTimeLocal'])
    return time_controls

In [23]:
if __name__=='__main__':
    time_controls = dbGetTimeControls(rally)
    display(time_controls.head())

,date,code,controlId,controlPenalties,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,...,targetDurationMs,timingPrecision,type,itineraryLegId,itinerarySections.itinerarySectionId,roundingPolicy,timeZoneOffset,itinerarySectionId,section,order
0,Friday 16th November,TC0,3503,All,NaN,38,2018-11-15T20:15:00,2018-11-15 20:15:00,Coffs Start,-1,...,NaN,Minute,TimeControl,146,338,RoundToClosestMinute,660,338,Section 1,1
1,Friday 16th November,TC1,3558,All,31.68,38,2018-11-15T21:00:00,2018-11-15 21:00:00,Orara East I,838,...,2700000.0,Minute,TimeControl,146,338,RoundToClosestMinute,660,338,Section 1,1
2,Friday 16th November,SS1,3557,None,8.77,38,2018-11-15T21:03:00,2018-11-15 21:03:00,Orara East I,838,...,180000.0,Minute,StageStart,146,338,RoundToClosestMinute,660,338,Section 1,1
3,Friday 16th November,TC2,3555,All,23.95,38,2018-11-15T21:40:00,2018-11-15 21:40:00,Coldwater I,840,...,2220000.0,Minute,TimeControl,146,338,NoRounding,660,338,Section 1,1
4,Friday 16th November,SS2,3554,None,14.12,38,2018-11-15T21:43:00,2018-11-15 21:43:00,Coldwater I,840,...,180000.0,Minute,StageStart,146,338,RoundToClosestMinute,660,338,Section 1,1


In [24]:
#Check datetime type
time_controls['firstCarDueDateTime'] = pd.to_datetime(time_controls['firstCarDueDateTime'])

In [25]:
import datetime

def newtime(row):
    t=datetime.timedelta( minutes=row['timeZoneOffset'])
    return row['firstCarDueDateTime']+ t

time_controls['mylocaltime'] = time_controls.apply(lambda row: newtime(row),axis=1)

In [26]:
rules = {'origin': "#mylocaltime.pdtime(%H:%M:%S)# #code# #location# #distance.isNotNull(post=km).brackets# \[#targetDuration#\]",
        }

In [27]:
dategroups = time_controls.groupby('date',sort=False)
for key in dategroups.groups.keys():
    print('---\n\n{}:\n'.format(key))
    grouped2=dategroups.get_group(key).groupby('section',sort=False)
    for key2 in grouped2.groups.keys():
        g2 = grouped2.get_group(key2)
        l=len(g2[g2['code'].str.startswith('SS')])
        print('{} - {} special {}\n'.format(key2,p.number_to_words(l),p.plural_noun('stage',l)))
        for r in grouped2.get_group(key2).apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            print('\t{}'.format(r))
        print('\n')

---

Friday 16th November:

Section 1 - three special stages

	07:15:00 TC0 Coffs Start  [None]
	08:00:00 TC1 Orara East I (31.68km) [00:45:00]
	08:03:00 SS1 Orara East I (8.77km) [00:03:00]
	08:40:00 TC2 Coldwater I (23.95km) [00:37:00]
	08:43:00 SS2 Coldwater I  (14.12km) [00:03:00]
	09:38:00 TC3 Sherwood I (22.42km) [00:55:00]
	09:41:00 SS3 Sherwood I (26.68km) [00:03:00]
	10:56:00 TC3A Coffs Regroup & Technical Zone IN (39.33km) [01:15:00]


Section 2 - three special stages

	11:14:00 TC3B Coffs Regroup OUT / Coffs Service IN  [00:18:00]
	11:44:00 TC3C Coffs Service OUT  [00:30:00]
	12:29:00 TC4 Orara East II (31.68km) [00:45:00]
	12:32:00 SS4 Orara East II (8.77km) [00:03:00]
	13:09:00 TC5 Coldwater II (23.95km) [00:37:00]
	13:12:00 SS5 Coldwater II (14.12km) [00:03:00]
	14:07:00 TC6 Sherwood II (22.42km) [00:55:00]
	14:10:00 SS6 Sherwood II (26.68km) [00:03:00]
	15:25:00 TC6A SSS Regroup IN (39.33km) [01:15:00]


Section 3 - two special stages

	16:22:00 TC6B SSS Regroup OUT  [00

In [28]:
def initStageReports(rebase='overall'):
    ''' Generate the initial report for a particular section rebased to a particular position or driver.'''
    sectionREADME = '''### {section} Report
    '''

    SUMMARY ='\n'
    
    rules['stages'] = "#code# - #location# #distance.isNotNull(post=km).brackets#"

    dn = '' if not rebase or 'overall' in rebase else '_'+rebase
  
    sections = time_controls.groupby('section',sort=False)
    keyorder = [k for k in time_controls.sort_values('order')['section'].unique() if k in sections.groups.keys()]
    for key in keyorder:

        sectionfn = '{bp}/{key}_report{dn}.md'.format(bp=basepath,key=key, dn=dn)

        with open(sectionfn, 'w') as out_file:
            out_file.write('')

        sectionControls = sections.get_group(key)
        sstages = sectionControls[sectionControls['code'].str.startswith('SS')]
        l=len(sstages)
        
        if l:
            title = '# {}, {}\n\nThis section comprises {} special {}'.format(key,sectionControls['date'].iloc[0], p.number_to_words(l),p.plural_noun('stage',l))
            sstage=[]
            for r in sstages.apply(lambda row: pandas_row_mapper(row, rules, "#stages#",[base_english,inflect_english]), axis=1):
                sstage.append(r)
            sectionREADME = '{} ({})'.format(sectionREADME,', '.join(sstage))
        else: 
            title = '# {}, {}\n\nThere were no special stages in this section.'.format(key,sectionControls['date'].iloc[0])
        sectionREADME = '''{title}'''.format( title=title)

        sectionREADME = '''{s}\n\nThe full scheduled itinerary for the section was as follows:\n'''.format(s=sectionREADME,)

        controls = []
        for r in sectionControls.apply(lambda row: pandas_row_mapper(row, rules, "#origin#",[base_english,inflect_english]), axis=1):
            controls.append(r)
        sectionREADME = '{}\n\t- {}\n'.format(sectionREADME, '\n\t- '.join(controls))

        print(sectionREADME,'\n------\n')
        with open(sectionfn, 'a') as out_file:
            out_file.write(sectionREADME)

        #Add section
        SUMMARY = '{summary}\n* [{key}]({key}_report{dn}.md)\n'.format(summary=SUMMARY, key=key,dn=dn)

        sstageDict = sstages.set_index('code').to_dict(orient='index')
        #Add special stages
        for s in sstageDict: #[Section 2](Section 2_report.md)
            SUMMARY = '{summary}  - [{s} - {n}]({s}_report{dn}.md)\n'.format(summary=SUMMARY,
                                                                              s=s, 
                                                                              n=sstageDict[s]['location'],
                                                                              dn=dn)

    with open('{}/SUMMARY{}.md'.format(basepath,dn), 'a') as out_file:
        out_file.write(SUMMARY)

initStageReports(rebase)
#initStageReports()

# Section 1, Friday 16th November

This section comprises three special stages

The full scheduled itinerary for the section was as follows:

	- 07:15:00 TC0 Coffs Start  [None]
	- 08:00:00 TC1 Orara East I (31.68km) [00:45:00]
	- 08:03:00 SS1 Orara East I (8.77km) [00:03:00]
	- 08:40:00 TC2 Coldwater I (23.95km) [00:37:00]
	- 08:43:00 SS2 Coldwater I  (14.12km) [00:03:00]
	- 09:38:00 TC3 Sherwood I (22.42km) [00:55:00]
	- 09:41:00 SS3 Sherwood I (26.68km) [00:03:00]
	- 10:56:00 TC3A Coffs Regroup & Technical Zone IN (39.33km) [01:15:00]
 
------

# Section 2, Friday 16th November

This section comprises three special stages

The full scheduled itinerary for the section was as follows:

	- 11:14:00 TC3B Coffs Regroup OUT / Coffs Service IN  [00:18:00]
	- 11:44:00 TC3C Coffs Service OUT  [00:30:00]
	- 12:29:00 TC4 Orara East II (31.68km) [00:45:00]
	- 12:32:00 SS4 Orara East II (8.77km) [00:03:00]
	- 13:09:00 TC5 Coldwater II (23.95km) [00:37:00]
	- 13:12:00 SS5 Coldwater II (14.12km) [

In [29]:
sstages['date']

NameError: name 'sstages' is not defined

In [ ]:
sstages.set_index('code').to_dict(orient='index')

In [41]:
print(SUMMARY)

NameError: name 'SUMMARY' is not defined

In [42]:
!head -n100 "report/Section 3_report.md"

# Section 3, Friday 26th October

This section comprises three special stages (SS2 - Gandesa 1 (gravel) (7.0km), SS3 - Pesells 1 (gravel) (26.59km), SS4 - La Fatarella -Vilalba 1 (gravel & asphalt) (38.85km))

The full scheduled itinerary for the section was as follows:

	- 08:00:00 TC1B Parc Fermé OUT - Service IN  [None]
	- 08:15:00 TC1C Service OUT  [00:15:00]
	- 09:30:00 TC2 Gandesa (79.93km) [01:15:00]
	- 09:33:00 SS2 Gandesa 1 (gravel) (7.0km) [00:03:00]
	- 10:03:00 TC3 Pesells (10.07km) [00:30:00]
	- 10:06:00 SS3 Pesells 1 (gravel) (26.59km) [00:03:00]
	- 11:17:00 TC4 La Fatarella (32.48km) [01:11:00]
	- 11:20:00 SS4 La Fatarella -Vilalba 1 (gravel & asphalt) (38.85km) [00:03:00]
	- 13:15:00 TC4A Regroup and Technical Zone  IN (85.37km) [01:55:00]


In [ ]:
sectionControls['code']

In [ ]:
sectionControls#[sectionControls['code']]

In [ ]:
l